# Imported Library

In [292]:
import psycopg2
from psycopg2 import sql
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone
from dateutil import tz

import os
import sys 

from configupdater import ConfigUpdater
# pip install ConfigUpdater

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

import bq_storage_api.incident_data_pb2 as pb2_incident

# [metadata]
# pmr_pm_plan = 2019-01-01 00:00:00
# pmr_pm_item = 2019-01-01 00:00:00
# pmr_project = 2019-01-01 00:00:00
# pmr_inventory  = 2019-01-01 00:00:00


In [293]:
way="bq-storage-api" # 1="merge"  or "bq-storage-api"
view_name = "xyz_incident"


# way="merge" 
# view_name = "pmr_pm_plan"

is_py=False



In [294]:
data_name=view_name.replace("pmr_","").replace("xyz_","")

                             
isFirstLoad=False
                             

if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        view_name=sys.argv[1]
    else:
        print("Enter the following input: ")
        view_name = input("View Table Name : ")
print(f"View name to load to BQ :{view_name} and data name: {data_name}")

View name to load to BQ :xyz_incident and data name: incident


# Imported date

In [295]:
dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
print(f"UTC: {dt_imported} For This Import")



UTC: 2024-01-11 10:35:56 For This Import


# Set view data and log table and protocolbuffers

In [296]:
log = "models_logging_change"

data_pb2=None
sp=f"merge_{data_name}"

def get_process_configuration_data(view_name):
    
    x_data_pb2=None
    if view_name == "pmr_pm_plan":
        tableContentID = 36
        key_name = "pm_id"
        changed_field_mapping=['planned_date','ended_pm_date',
                               'project_id','remark','team_lead_id']

    elif view_name == "pmr_pm_item":
        tableContentID = 37
        key_name = "pm_item_id"


    elif view_name == "pmr_project":
        tableContentID = 7
        key_name = "project_id"


    elif view_name == "pmr_inventory":
        tableContentID = 14
        key_name = "inventory_id"

        
    elif view_name == "xyz_incident":
        tableContentID = 18
        key_name = "incident_id"   
        x_data_pb2=pb2_incident.IncidentData()

    else:
        raise Exception("No specified content type id")
        
    return tableContentID, key_name,sp,x_data_pb2


                             
                               
content_id , view_name_id,sp_name,data_pb2=get_process_configuration_data(view_name)
print(content_id," - ",view_name_id)

18  -  incident_id


# Check configuration parameter validation:

In [297]:
#  # 1="merge"  or "bq-storage-api"
def check_config_parameter_validation(way,sp,data_pb2):
  if  (way=="merge") and sp is None:
     raise Exception(f"StoreProcedure is not allowed to None in {way} Way.")
  elif  (way=="bq-storage-api") and data_pb2 is None:
     raise Exception(f"ProtoBuf Data is not allowed to None in {way} Way.")   
  return True
    
result_data_validation=check_config_parameter_validation(way,sp,data_pb2)
if result_data_validation and  way=="merge":
 print(f"{way} - {sp}")
elif result_data_validation and  way=="bq-storage-api":
 print(f"{way}")
 print(data_pb2.DESCRIPTOR)

bq-storage-api


# Set data and cofig path

In [298]:
# Test config,env file and key to be used ,all of used key  are existing.
cfg_path="cfg_last_import"
env_path='.env'

updater = ConfigUpdater()
updater.read(os.path.join(cfg_path,f"{view_name}.cfg"))

config = dotenv_values(dotenv_path=env_path)

print(env_path)
print(cfg_path)

.env
cfg_last_import


In [299]:
# Test exsitng project dataset and table anme

projectId=config['PROJECT_ID']  # smart-data-ml  or kku-intern-dataai or ponthorn
credential_file=config['PROJECT_CREDENTIAL_FILE']
# C:\Windows\smart-data-ml-91b6f6204773.json
# C:\Windows\kku-intern-dataai-a5449aee8483.json
# C:\Windows\pongthorn-5decdc5124f5.json


dataset_id='SMartData_Temp'  # 'SMartData_Temp'  'PMReport_Temp'
main_dataset_id='SMartDataAnalytics'  # ='SMartDataAnalytics'  'PMReport_Main'

credentials = service_account.Credentials.from_service_account_file(credential_file)

table_name=f"temp_{data_name}" #can change in ("name") to temp table
table_id = f"{projectId}.{dataset_id}.{table_name}"
print(table_id)


main_table_name=data_name
main_table_id = f"{projectId}.{main_dataset_id}.{main_table_name}"
print(main_table_id)

# https://cloud.google.com/bigquery/docs/reference/rest/v2/Job
to_bq_mode="WRITE_EMPTY"


client = bigquery.Client(credentials= credentials,project=projectId)

pongthorn.SMartData_Temp.temp_incident
pongthorn.SMartDataAnalytics.incident


Read Configuration File and Initialize BQ Object

In [300]:
last_imported=datetime.strptime(updater["metadata"][view_name].value,"%Y-%m-%d %H:%M:%S")
print(f"{data_name} - UTC:{last_imported}  Of Last Import")

# local_zone = tz.tzlocal()
# last_imported = last_imported.astimezone(local_zone)
# print(f"Local Asia/Bangkok:{last_imported}")

incident - UTC:2024-01-01 00:00:00  Of Last Import


# Postgres &BigQuery

In [301]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [302]:
def get_bq_table():
 try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
    return True
 except NotFound:
    raise Exception("Table {} is not found.".format(table_id))
    

def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(write_disposition=to_bq_mode,)
        job = client.load_table_from_dataframe(df_trasns, table_id, job_config=job_config)
        try:
         job.result()  # Wait for the job to complete.
        except ClientError as e:
         print(job.errors)

        print("Total ", len(df_trasns), f"Imported data to {table_id} on bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

# Check whether it is the first loading?

In [303]:
def checkFirstLoad():
    print("If the main table is empty , so the action of each row  must be 'added' on temp table")
    rows_iter   = client.list_rows(main_table_id, max_results=1) 
    no_main=len(list(rows_iter))
    if no_main==0:
     isFirstLoad=True
     print(f"This is the first loaing , so there is No DATA in {main_table_id}, we load all rows from {view_name} to import into {table_id} action will be 'added' ")
    else:
     isFirstLoad=False   
    return isFirstLoad

In [304]:
isFirstLoad=checkFirstLoad()
print(f"IsFirstLoad={isFirstLoad} for {data_name}")

If the main table is empty , so the action of each row  must be 'added' on temp table
This is the first loaing , so there is No DATA in pongthorn.SMartDataAnalytics.incident, we load all rows from xyz_incident to import into pongthorn.SMartData_Temp.temp_incident action will be 'added' 
IsFirstLoad=True for incident


# For The next Load
* Get data from model log based on condition last_imported and table
* Get all actions from log table by selecting unique object_id and setting by doing something as logic
* Create  id and action dataframe form filtered rows from log table

In [305]:
def list_model_log(x_last_imported,x_content_id):
    sql_log = f"""
    SELECT object_id, action,TO_CHAR(date_created,'YYYY-MM-DD HH24:MI:SS') as date_created ,changed_data
    FROM {log}
    WHERE date_created  AT time zone 'utc' >= '{x_last_imported}' AND content_type_id = {x_content_id} 
    ORDER BY object_id, date_created
    """
    print(sql_log)


    # Asia/Bangkok 
    lf = list_data(sql_log, None, get_postgres_conn())
    print(f"Retrieve all rows after {last_imported}")
    print(lf.info())
    return lf


In [306]:
def check_any_changes_to_collumns_view(dfAction,x_view_name,_x_key_name):
    """
    Check dataframe from log model that contain only changed action to select changed fields on view.
    """

    listACtion=dfAction["action"].unique().tolist()
    if len(listACtion)==1 and listACtion[0]=='changed':
        print("###########################################################")
        print("Process dataframe containing only all changed action")
        print(dfAction)
        print("###########################################################")
    
    

In [307]:
def select_actual_action(lf):
    listIDs=lf["object_id"].unique().tolist()
    listUpdateData=[]
    for id in listIDs:
        lfTemp=lf.query("object_id==@id")
        print(f"--------------------{id}---------------------------------")
        print(lfTemp)
        
        
        
        # check_any_changes_to_collumns_view(lfTemp,content_id,view_name_id)


        first_row = lfTemp.iloc[0]
        last_row = lfTemp.iloc[-1]
        # print(first_row)
        # print(last_row)

        if len(lfTemp)==1:
            listUpdateData.append([id,first_row["action"]])
        else:
            if first_row["action"] == "added" and last_row["action"] == "deleted":
                continue
            elif first_row["action"] == "added" and last_row["action"] != "deleted":
                listUpdateData.append([id,"added"])
            else : listUpdateData.append([id,last_row["action"]])

    print("Convert listUpdate to dataframe")
    dfUpdateData = pd.DataFrame(listUpdateData, columns= ['id', 'action'])
    dfUpdateData['id'] = dfUpdateData['id'].astype('int64')
    dfUpdateData=dfUpdateData.sort_values(by="id")
    dfUpdateData=dfUpdateData.reset_index(drop=True)

    return dfUpdateData


In [308]:
if isFirstLoad==False:
    listModelLogObjectIDs=[]
    dfModelLog=list_model_log(last_imported,content_id)
    if dfModelLog.empty==True:
        print("No row to be imported.")
        exit()
    else:
       print("Get row imported from model log to set action") 
       dfModelLog=select_actual_action( dfModelLog)
       listModelLogObjectIDs=dfModelLog['id'].tolist()
       print(dfModelLog.info())
       print(dfModelLog)       
       print(listModelLogObjectIDs) 

# Load view and transform

In [309]:
def retrive_next_data_from_view(x_view,x_id,x_listModelLogObjectIDs):
    if len(x_listModelLogObjectIDs)>1:
     sql_view=f"select *  from {x_view}  where {x_id} in {tuple(x_listModelLogObjectIDs)}"
    else:
     sql_view=f"select *  from {x_view}  where {x_id} ={x_listModelLogObjectIDs[0]}"
    
    print(sql_view)
    df=list_data(sql_view,None,get_postgres_conn())

    if df.empty==True:
     return df
    df=df.drop(columns='updated_at')
    return df 


def retrive_first_data_from_view(x_view,x_last_imported):
     sql_view=f"select *  from {x_view}  where  updated_at AT time zone 'utc' >= '{x_last_imported}'"
     print(sql_view)
     df=list_data(sql_view,None,get_postgres_conn())
     if df.empty==True:
            return df
     df=df.drop(columns='updated_at')
     df['action']='added'
     return df   
def retrive_one_row_from_view_to_gen_df_schema(x_view):
    sql_view=f"select *  from {x_view}  limit 1"
    print(sql_view)
    df=list_data(sql_view,None,get_postgres_conn())
    df=df.drop(columns='updated_at')
    return df


if isFirstLoad:
 df=retrive_first_data_from_view(view_name,last_imported)
 if df.empty==True:
    print("No row to be imported.")
    exit()
 else:
    print(df)

else:
 df=retrive_next_data_from_view(view_name,view_name_id,listModelLogObjectIDs)  
 if df.empty==True:
    print("Due to having deleted items, we will Get schema from {} to create empty dataframe with schema.")
    df=retrive_one_row_from_view_to_gen_df_schema(view_name)
    # this id has been included in listModelLogObjectIDs which contain deleted action , so we can use it as schema generation
    print(df)

    
print(df.info())    
    

select *  from xyz_incident  where  updated_at AT time zone 'utc' >= '2024-01-01 00:00:00'
   incident_id  inventory_id            incident_type service_type  severity  \
0         4302          9594  Network Adapter Failure     Incident     Minor   
1         4303         17974         General Incident      Request  Cosmetic   
2         4298         12306         General Incident     Incident     Major   
3         4299         12306         General Incident     Incident     Major   
4         4301         19244       Maintenance System      Request  Cosmetic   
5         4304         12305         General Incident     Incident     Major   
6         4305         19211                 Software     Incident     Major   
7         4300           993                 Software     Incident     Major   

        status        open_datetime       close_datetime action  
0         Open  2023-12-27 09:55:00  2024-01-03 02:00:00  added  
1  In Progress  2024-01-03 09:00:00  2024-01-03 12:00:00

# Data Transaformation
* IF The first load then add actio='Added'
* IF The nextload then Merge LogDF and ViewDF and add deleted row 
  * Get Deleted Items  to Create deleted dataframe by using listDeleted
  * If there is one deletd row then  we will merge it to master dataframe
* IF the next load has only deleted action

In [310]:
def add_acutal_action_to_df_at_next(df,dfUpdateData,x_view,x_id):
    # merget model log(id and action) to data view
    # if  dfUpdateData  contain only deleted action
    # we will merge to get datafdame shcema, it can perform inner without have actual data fram view
    merged_df = pd.merge(df, dfUpdateData, left_on=view_name_id, right_on='id', how='inner')
    merged_df = merged_df.drop(columns=['id'])

    listAllAction=dfUpdateData['id'].tolist()
    print(f"List {listAllAction} all action")
    
    listSeleted = merged_df[view_name_id].tolist()
    print(f"List  {x_view}   {listSeleted} from {x_view} exluding deleted action")
    
    allActionSet = set(listAllAction)
    anotherSet = set(listSeleted)
    
    listDeleted = list(allActionSet.symmetric_difference(anotherSet))
    print(f"List deleted {listDeleted}")
    
    # Test List  select by view + List deeleted = List All Action

    if len(listDeleted)>0:
        print("There are some deleted rows")
        dfDeleted=pd.DataFrame(data=listDeleted,columns=[view_name_id])
        dfDeleted['action']='deleted'
        print(dfDeleted)
        merged_df=pd.concat([merged_df,dfDeleted],axis=0)

    else:
        print("No row deleted")

    return merged_df    




In [311]:
if isFirstLoad==False:
 df=add_acutal_action_to_df_at_next(df,dfModelLog,view_name,view_name_id)

print(df)

   incident_id  inventory_id            incident_type service_type  severity  \
0         4302          9594  Network Adapter Failure     Incident     Minor   
1         4303         17974         General Incident      Request  Cosmetic   
2         4298         12306         General Incident     Incident     Major   
3         4299         12306         General Incident     Incident     Major   
4         4301         19244       Maintenance System      Request  Cosmetic   
5         4304         12305         General Incident     Incident     Major   
6         4305         19211                 Software     Incident     Major   
7         4300           993                 Software     Incident     Major   

        status        open_datetime       close_datetime action  
0         Open  2023-12-27 09:55:00  2024-01-03 02:00:00  added  
1  In Progress  2024-01-03 09:00:00  2024-01-03 12:00:00  added  
2         Open  2023-12-30 12:08:00  2023-12-30 13:00:00  added  
3         Open 

# Last Step :Check duplicate ID & reset index

In [312]:
hasDplicateIDs = df[view_name_id].duplicated().any()
if  hasDplicateIDs:
 raise Exception("There are some duplicate id on dfUpdateData")
else:
 print(f"There is no duplicate {view_name_id} ID")  


# merged_df['imported_at']=dt_imported
df=df.reset_index(drop=True  )
print(df.info())
print(df)

There is no duplicate incident_id ID
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     8 non-null      int64 
 1   inventory_id    8 non-null      int64 
 2   incident_type   8 non-null      object
 3   service_type    8 non-null      object
 4   severity        8 non-null      object
 5   status          8 non-null      object
 6   open_datetime   8 non-null      object
 7   close_datetime  5 non-null      object
 8   action          8 non-null      object
dtypes: int64(2), object(7)
memory usage: 704.0+ bytes
None
   incident_id  inventory_id            incident_type service_type  severity  \
0         4302          9594  Network Adapter Failure     Incident     Minor   
1         4303         17974         General Incident      Request  Cosmetic   
2         4298         12306         General Incident     Incident     Major   

# Insert data to BQ data frame & # Run StoreProcedure To Merge Temp&Main and Truncate Transaction 

In [313]:
if way=='merge':
    print("1#Ingest data into Bigquery")
    if get_bq_table():
        try:
            insertDataFrameToBQ(df)
        except Exception as ex:
            raise ex
            
    print("2#Run StoreProcedure To Merge Temp&Main and Truncate Transaction.")
    # https://cloud.google.com/bigquery/docs/transactions
    sp_id_to_invoke=f""" CALL `{projectId}.{main_dataset_id}.{sp_name}`() """
    print(sp_id_to_invoke)    
    sp_job = client.query(sp_id_to_invoke)

else:
    bq_storage_api_path="bq_storage_api"
    df.to_csv(f"{data_name}_{way}.csv",index=False)

# BQ-Storage-API Data Transformation

In [314]:
print("convert strng to datetime and microseconds")
from google.protobuf.timestamp_pb2 import Timestamp
def convert_string_to_datetime_timestamp_microseconds (dt_str):
    if dt_str is not None:   
        dt=datetime.strptime(dt_str,"%Y-%m-%d %H:%M:%S")
        x_timestamp = Timestamp()
        x_timestamp.FromDatetime(dt)
        micro_x =x_timestamp.ToMicroseconds()
        return micro_x
    else:
        None
#        
datetimeCols=["open_datetime","close_datetime"]
for d in datetimeCols:
    # check whick column contain null value if so, convert float64 to int 32
    df[d]=df[d].apply(convert_string_to_datetime_timestamp_microseconds)
    df[d] = df[d].fillna(0)
    df[d]=df[d].astype('Int64')

convert strng to datetime and microseconds


In [315]:
timestamp = Timestamp()
timestamp.FromDatetime(dt_imported)
update_at_micro_timestampe =timestamp.ToMicroseconds()
df['update_at']=update_at_micro_timestampe 


In [316]:
print("change action type")

def change_action_merge_to_bq_storage_api(x):
    if x=="added" or x=="changed":
        return  "UPSERT"
    else:
        return "DELETE"

    
df["_CHANGE_TYPE"]=df['action'].apply(change_action_merge_to_bq_storage_api)
df=df.drop(columns=['action'])


change action type


In [317]:
print(df.info())
df.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     8 non-null      int64 
 1   inventory_id    8 non-null      int64 
 2   incident_type   8 non-null      object
 3   service_type    8 non-null      object
 4   severity        8 non-null      object
 5   status          8 non-null      object
 6   open_datetime   8 non-null      Int64 
 7   close_datetime  8 non-null      Int64 
 8   update_at       8 non-null      int64 
 9   _CHANGE_TYPE    8 non-null      object
dtypes: Int64(2), int64(3), object(5)
memory usage: 784.0+ bytes
None


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
3,4299,12306,General Incident,Incident,Major,Open,1703960400000000,1703962800000000,1704969356000000,UPSERT
4,4301,19244,Maintenance System,Request,Cosmetic,In Progress,1704232140000000,0,1704969356000000,UPSERT
5,4304,12305,General Incident,Incident,Major,Open,1704275040000000,1704277200000000,1704969356000000,UPSERT
6,4305,19211,Software,Incident,Major,Open,1704672000000000,0,1704969356000000,UPSERT
7,4300,993,Software,Incident,Major,Open,1704200400000000,0,1704969356000000,UPSERT


In [321]:
# df.to_csv(f"{data_name}_{way}.csv",index=False)
json_file="incident.json"
json_file_path=os.path.join(bq_storage_api_path,json_file)

json_incident_data = json.loads(df.to_json(orient = 'records'))
with open(json_file_path, "w") as outfile:
    json.dump(json_incident_data, outfile)



# Update New Recenet Update to file


In [459]:
updater["metadata"][view_name].value=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
updater.update_file() 

<ConfigUpdater [
    <Section: 'metadata' [
        <Option: xyz_incident = '2024-01-11 08:34:17'>
    ]>
]>

In [460]:
print(datetime.now(timezone.utc) )

2024-01-11 08:34:29.591782+00:00
